<a href="https://colab.research.google.com/github/AstxMargaryan/ML/blob/main/SVMs_and_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SVMs

### Homework: Implement a Linear SVM (Classification)

In this assignment, you will implement a **linear Support Vector Machine (SVM)** for binary classification.  
You are given the class skeleton below; your task is to complete the `fit` and `predict` methods.  
Assume labels are binary and convert them internally to \(\{-1, +1\}\). The goal is to correctly learn the weight vector \(w\) and bias \(b\) and use them to make predictions.


In [ ]:
import numpy as np


class SVM:

    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None


    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_signed = np.where(y<=0, -1, 1)

        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
          for i in range(n_samples):
            margin = y_signed[i] * (np.dot(self.w, X[i]) + self.b)

            if margin >= 1:
              self.w -= self.lr * (2 * self.lambda_param * self.w)
            else:
              self.w -= self.lr * (2 * self.lambda_param * self.w - y_signed[i] * X[i])
              self.b -= self.lr * (-y_signed[i])


    def predict(self, X):
        scores = np.dot(X, self.w) + self.b
        y_pred = np.where(scores >= 0, 1, -1)
        return y_pred

## Decision Trees

### Homework: Implement a Decision Tree Classifier

In this assignment, you will implement a **basic decision tree classifier from scratch**.  
The tree should use the **Gini impurity** to choose splits and should grow recursively.

Stop splitting when a maximum depth is reached **or** when a node contains too few samples.  
You do **not** need to implement preprocessing, feature encoding, or pruning.


In [ ]:
import numpy as np

class DecisionTreeClassifier:
    def __init__(self, max_depth=3, min_size=5):
        self.max_depth = max_depth
        self.min_size = min_size
        self.tree = None

    def gini(self, groups, classes):
      """Compute Gini impurity for a split."""
      n_instances = sum(len(group) for group in groups)
      if n_instances == 0:
        return 0.0

      gini = 0.0
      for group in groups:
        size = len(group)
        if size == 0:
          continue

        score = 0.00
        labels = group[:, -1]
        for c in classes:
          p = np.sum(labels == c) / size
          score += p * p
        gini += (1-score) * (size / n_instances)
      return gini

    def split_dataset(self, index, value, dataset):
      """Split dataset based on feature index and threshold value."""
      left = dataset[dataset[:, index] < value]
      right = dataset[dataset[:, index] >= value]
      return left, right

    def best_split(self, dataset):
      """Find the best split for a dataset."""
      classes = np.unique(dataset[:, -1])
      n_features = dataset.shape[1] - 1

      best_index, best_value, best_score, best_groups = None, None, float("inf"), None

      for index in range(n_features):
        for row in dataset:
          value = row[index]
          groups = self.split_dataset(index, value, dataset)
          gini = self.gini(groups, classes)
          if gini < best_score:
            best_index, best_value, best_score, best_groups = index, value, gini, groups

      return {"index": best_index, "value": best_value, "groups": best_groups}

    def to_terminal(self, group):
      """Create a terminal node (most common class)."""
      labels = group[:, -1]
      values, counts = np.unique(labels, return_counts=True)
      return values[np.argmax(counts)]

    def build_tree(self, node, depth):
      """Recursively build the decision tree."""
      left, right = node['groups']
      del node['groups']

      if len(left) == 0 or len(right) == 0:
        node['left'] = node['right'] = self.to_terminal(np.vstack((left, right)))
        return

      if depth >= self.max_depth:
        node['left'], node['right'] = self.to_terminal(left), self.to_terminal(right)
        return

      if len(left) <= self.min_size:
        node['left'] = self.to_terminal(left)
      else:
        node['left'] = self.best_split(left)
        self.build_tree(node['left'], depth + 1)

      if len(right) <= self.min_size:
        node['right'] = self.to_terminal(right)
      else:
        node['right'] = self.best_split(right)
        self.build_tree(node['right'], depth + 1)


    def fit(self, X, y):
        """Build the decision tree."""
        dataset = np.column_stack((X, y))
        root = self.best_split(dataset)
        self.build_tree(root, depth=1)
        self.tree = root

    def predict_one(self, node, x):
      """Predict class for a single sample."""
      if not isinstance(node, dict):
        return node
      if x[node['index']] < node['value']:
        return self.predict_one(node['left'], x)
      else:
        return self.predict_one(node['right'], x)


    def predict(self, X):
        """Predict classes for multiple samples."""
        return np.array([self.predict_one(self.tree, x) for x in X])
